In [2]:
# load libraries
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [6]:
# read training and test sets
data=np.load("FashionData/FashionPDEngDM.npz")

##Labeled training set for classes 1,2,3,8,9 (30000 samples)
x_train_12389_labeled=data["x_train_12389_labeled"]
y_train_12389_labeled=data["y_train_12389_labeled"]

##Labeled training set for classes 0,4,5,6,7 (just 5 samples)
x_train_04567_labeled=data["x_train_04567_labeled"]
y_train_04567_labeled=data["y_train_04567_labeled"]

##Unlabeled training set for classes 0,4,5,6,7 (29992 samples)
x_train_04567_unlabeled=data["x_train_04567_unlabeled"]

##Labeled test set for classes 1,2,3,8,9
x_test_12389=data["x_test_12389"]
y_test_12389=data["y_test_12389"]

##Labeled test set for classes 0,4,5,6,7 (this is where we are interested to obtain the highest accuracy possible - project goal)
x_test_04567=data["x_test_04567"]
y_test_04567=data["y_test_04567"]


## Example of trainining a CNN on classes 1,2,3,8,9
# input image dimensions
img_rows, img_cols = 28, 28
#set CNN parameters
batch_size = 100
num_classes = 10
epochs = 5
FC_SIZE = 256

In [7]:
def preprocess_data(X, y):
    #create tensor variant of 2D images
    if K.image_data_format() == 'channels_first':
        X = X.reshape(X.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        X = X.reshape(X.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

    X = X.astype('float32') / 255.

    # convert class vectors to binary class matrices
    y = keras.utils.to_categorical(y, num_classes)
    return X, y, input_shape
x_train_12389_labeled, y_train_12389_labeled, input_shape = preprocess_data(x_train_12389_labeled, y_train_12389_labeled)
x_test_12389, y_test_12389, _ = preprocess_data(x_test_12389, y_test_12389)

x_train_04567_labeled, y_train_04567_labeled, input_shape = preprocess_data(x_train_04567_labeled, y_train_04567_labeled)
x_test_04567, y_test_04567, _ = preprocess_data(x_test_04567, y_test_04567)

In [8]:
def train_base_model(model_name):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    model.fit(x_train_12389_labeled,y_train_12389_labeled,
              batch_size=batch_size,
              epochs=epochs,
              shuffle=True,
              verbose=1,
              validation_data=(x_test_12389, y_test_12389))

    score = model.evaluate(x_test_12389, y_test_12389, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    model.save(model_name)
train_base_model(model_name = 'model_data.h5')

Train on 30000 samples, validate on 5000 samples
Epoch 1/5
30000/30000 [==============================] - 73s - loss: 0.1549 - acc: 0.9527 - val_loss: 0.0698 - val_acc: 0.9800
Epoch 2/5
30000/30000 [==============================] - 69s - loss: 0.0671 - acc: 0.9794 - val_loss: 0.0590 - val_acc: 0.9816
Epoch 3/5
30000/30000 [==============================] - 68s - loss: 0.0541 - acc: 0.9838 - val_loss: 0.0579 - val_acc: 0.9830
Epoch 4/5
30000/30000 [==============================] - 70s - loss: 0.0468 - acc: 0.9854 - val_loss: 0.0513 - val_acc: 0.9846
Epoch 5/5
30000/30000 [==============================] - 67s - loss: 0.0403 - acc: 0.9878 - val_loss: 0.0614 - val_acc: 0.9824
Test loss: 0.0613797692063
Test accuracy: 0.9824


In [6]:
# load saved based model
base_model = keras.models.load_model('model_data.h5')

In [19]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [24]:
# preparing transfer model
for layer in base_model.layers:
    layer.trainable = False #turn of all layers for training

# adding a dense layer behind flatten_1 layers (i.e., layer[-4])
x = Dense(128, activation='relu', name = "dense_new1")(base_model.layers[-4].output)  
x = Dropout(0.5, name='dropout_new')(x)  
predictions = Dense(num_classes, activation='softmax', name='softmax_new1')(x)  # new softmax layer
transfer_model = Model(input=base_model.input, output=predictions)
transfer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_new1 (Dense)           (None, 128)               1179776   
__________

C:\Users\azika\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("so..., inputs=Tensor("co...)`


In [25]:
# training transfer model
transfer_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='rmsprop',
              metrics=['accuracy'])

transfer_model.fit(x_train_04567_labeled, y_train_04567_labeled,
          batch_size=batch_size,
          epochs=epochs,
          shuffle=True,
          verbose=1,
          validation_data=(x_test_04567, y_test_04567))

score = transfer_model.evaluate(x_test_04567, y_test_04567, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
transfer_model.save('transfer_model.h5')

Train on 5 samples, validate on 5000 samples
Epoch 1/1
5/5 [==============================] - 3s - loss: 2.2683 - acc: 0.4000 - val_loss: 1.9272 - val_acc: 0.5080
Test loss: 1.92724760113
Test accuracy: 0.508
